# General imports

In [ ]:
import numpy as np
from glob import glob
from pathlib import Path

from multiview_prediction_toolkit.cameras.derived_cameras import MetashapeCameraSet
from multiview_prediction_toolkit.meshes import TexturedPhotogrammetryMesh
from multiview_prediction_toolkit.config import DATA_FOLDER

from multiview_prediction_toolkit.segmentation import (
    LookUpSegmentor,
    SegmentorPhotogrammetryCameraSet,
)

from multiview_prediction_toolkit.config import (
    EXAMPLE_CAMERAS_FILENAME,
    EXAMPLE_MESH_FILENAME,
)

# Processing parameters

In [ ]:
# Processing parameters
HEIGHT_ABOVE_GROUND_THRESH = 2
MESH_DOWNSAMPLE_TARGET = 0.25
AGGREGATE_IMAGE_SCALE = 0.25
BUFFER_RADIUS_METER = 50
NUM_CLASSES = 10

CAMERAS_FILENAME = EXAMPLE_CAMERAS_FILENAME
MESH_FILENAME = EXAMPLE_MESH_FILENAME
IMAGE_FOLDER = EXAMPLE_IMAGE_FOLDER
LABELS_FILENAME = EXAMPLE_LABELS_FILENAME
PREDICTED_IMAGE_LABELS_FOLDER = EXAMPLE_PREDICTED_IMAGE_LABELS_FOLDER

# Load the mesh

In [ ]:
mesh = TexturedPhotogrammetryMesh(MESH_FILENAME)

# Load the camera set and subsample

In [ ]:
# Create camera set
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
training_camera_set = camera_set.get_subset_near_geofile(
    LABELS_FILENAME, buffer_radius_meters=BUFFER_RADIUS_METER
)

In [ ]:
mesh.vis(camera_set=camera_set, force_xvfb=True)

In [ ]:
segmentor = LookUpSegmentor(
    base_folder=IMAGE_FOLDER,
    lookup_folder=PREDICTED_IMAGE_LABELS_FOLDER,
    num_classes=NUM_CLASSES,
)

segmentor_camera_set = SegmentorPhotogrammetryCameraSet(
    training_camera_set, segmentor=segmentor
)

In [ ]:
averaged_label_IDs, _, _ = mesh.aggregate_viewpoints_pytorch3d(
    segmentor_camera_set,
    image_scale=AGGREGATE_IMAGE_SCALE,
)

In [ ]:
max_class = np.argmax(averaged_label_IDs, axis=1, keepdims=True)
mesh.vis(vis_scalars=max_class)

# Export the prediction to a vector file

# Compare the quality of the prediction with the ground truth